In [1]:
import os
os.chdir('..')

os.environ["DJANGO_SETTINGS_MODULE"] = "course_management.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ["IS_LOCAL"] = "1"

import django
django.setup()

IS_LOCAL=True
['C:\\Users\\alexe\\.virtualenvs\\course-management-platform-wiAsnpQu\\Lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\alexe\\AppData\\Roaming\\jupyter\\runtime\\kernel-9eb5d2ed-4b55-4182-b34d-d65b8ae9db36.json']
Is test: False


In [ ]:
from collections import Counter
import pandas as pd

In [2]:
from courses.models import *

In [3]:
course = Course.objects.get(id=5)
print(course)

Machine Learning Zoomcamp 2024


In [4]:
output = r'C:/Users/alexe/git/zoomcamp-scoring/courses/mlzoomcamp-2024/graduates.csv'

In [25]:
min_projects = 2

In [5]:
cohort = 2024
course_name = 'ml-zoomcamp'
certificate_url_template = f"https://certificate.datatalks.club/{course_name}/{cohort}/{{hash}}.pdf"
certificate_url_template

'https://certificate.datatalks.club/ml-zoomcamp/2024/{hash}.pdf'

In [34]:
graduates = ProjectSubmission.objects \
    .filter(project__course=course, passed=True) \
    .prefetch_related("enrollment")

len(graduates)

393

In [35]:
cnt = Counter()
ids_mapping = {}

for g in graduates:
    e = g.enrollment
    eid = e.id
    cnt[eid] += 1
    ids_mapping[eid] = e

passed = []

for eid, c in list(cnt.items()):
    if c >= min_projects:
        passed.append(ids_mapping[eid])

In [36]:
from hashlib import sha1

def compute_hash(email):
    return sha1(email.encode('utf-8')).hexdigest()

def compute_certificate_id(email):
    email_clean = email.lower().strip()
    return compute_hash(email_clean + '_')

In [38]:
results = []

results.append({
    'email': 'never.give.up@gmail.com',
    'name': 'Rick Astley',
    'hash': 'na'
})

In [40]:
for enrollment in passed:
    student = enrollment.student

    email = student.email
    name = enrollment.certificate_name or enrollment.display_name
    hash = compute_certificate_id(email)

    url = certificate_url_template.format(hash=hash)
    enrollment.certificate_url = url
    
    results.append({
        'email': email,
        'name': name,
        'hash': hash
    })

In [41]:
df = pd.DataFrame(results)
df.to_csv(output, index=False)

In [43]:
Enrollment.objects.bulk_update(passed, fields=['certificate_url'])

170